In [2]:
import transformers
nlp = transformers.pipeline("conversational", 
                            model="microsoft/DialoGPT-medium")
#Time to try it out
input_text = "hello!"
nlp(transformers.Conversation(input_text), pad_token_id=50256)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at microsoft/DialoGPT-medium.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Conversation id: 419bab52-5f0f-4728-a26d-b4ae37f958d9
user: hello!
assistant: Hello! :D

In [3]:
while True:
    input_text=input('User: ')
    if input_text.lower()=='bye':
        print('GoodBye')
        break
    print(nlp(transformers.Conversation(input_text), pad_token_id=50256))
    
        

User: Yoooo!


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Conversation id: 680c3976-cb87-4488-95a7-76e4b95d3d49
user: Yoooo!
assistant: I'm so excited!

User: let's go mate


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Conversation id: 5142a828-c20e-4200-9324-b0a3f37f37bd
user: let's go mate
assistant: I'm not your mate, buddy.

User: why so rude


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Conversation id: d124a3f2-b894-4716-b56d-a334322ea09d
user: why so rude
assistant: I'm not rude, I'm just a little bit of a wuss.

User: bye
GoodBye


In [29]:
import pandas as pd
df=pd.read_excel('AttachmentMIT-Academic_Calendar_2023-24.-1 (2).xlsx')

In [30]:
df.drop([0,1],inplace=True)
df.columns=df.iloc[0]

In [34]:
df.drop(2,inplace=True)

In [36]:
df

2,NaN,July -23,Aug -23,Sept -23,Oct -23,Nov -23,Dec -23,NaN,Jan -24,Feb -24,Mar -24,Apr -24,May -24,Jun -24,July -24
3,NaN,1 S,1 T,1 F,1 SUNDAY,1 W Kannada Rajyotsava,1 F,NaN,1M,1 TH,1 F,Last Date to\nApply for\nBranch Change\nUtsav,1 W,1 S,1 M
4,NaN,2 SUNDAY,2 W,2 S,2 M Gandhi Jayanthi,NaN,Publishing list of\nS unsuccessful\nstudents i...,NaN,2T,NaN,NaN,NaN,Publishing list of\n2 TH unsuccessful\nstudent...,NaN,2 T
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 F,Applying for Branch\nChange starts,NaN,NaN,2 SUNDAY,NaN
6,NaN,NaN,NaN,NaN,NaN,2 TH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,3 M,3 TH,3 SUNDAY,NaN,NaN,NaN,NaN,"II, IV, VI Semester\nB. Tech./II Semester\nM.T...",NaN,NaN,NaN,NaN,NaN,3 W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,NaN,NaN,NaN,30S Mid-term Exam,NaN,NaN,NaN,NaN,30 T,NaN,30 S,NaN,30 TH,NaN,NaN
129,NaN,NaN,NaN,NaN,30M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30 T
130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31 W,29 TH,NaN,30T,31 F,30 S,NaN
131,NaN,NaN,NaN,NaN,31 T,30TH,31SUNDAY,NaN,NaN,NaN,31 SUNDAY,NaN,NaN,NaN,31 W


In [80]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
while True:
    input_text=input('User: ')
    if input_text.lower()=='bye':
        print('GoodBye')
        break
    for i in df.select_dtypes(include='object').columns:
        for j in df[i]:
            if type(j)!=float:
                v=TfidfVectorizer()
                vectors=v.fit_transform([j.lower(),input_text.lower()])
                sim=cosine_similarity(vectors)[0][1]
                if sim>0.2:
                    print(f'{i} : {j}')
    
    print(nlp(transformers.Conversation(input_text), pad_token_id=50256))

User: hello


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Conversation id: df9dfd0f-c0f1-4b1a-ac3c-9a53b764a8f8
user: hello
assistant: Hello! :D

User: holiday
July -23 : 15S Holiday
Aug -23 : 19S Holiday
Sept -23 : 16S Holiday
Nov -23 : 18S Holiday
Dec -23 : 16S Holiday
Jan -24 : 20S Holiday


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Feb -24 : 17 S Holiday
Mar -24 : 13 W
14 TH Revels
15 F
Holiday
Revels
Apr -24 : 20S Holiday
May -24 : 18 S Holiday
Jun -24 : 15 S Holiday
July -24 : 20 S Holiday
Conversation id: bde222df-51a5-41a4-b447-cbe5df08107f
user: holiday
assistant: Holiday

User: when do i have a holiday?
July -23 : 15S Holiday
Aug -23 : 19S Holiday
Sept -23 : 16S Holiday
Nov -23 : 18S Holiday
Dec -23 : 16S Holiday
Jan -24 : 20S Holiday


...

Conversation id: 581f6aa6-8215-4692-b158-52ad908a63d0
user: diwali vacation?
assistant: I wish!

User: diwali?


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Conversation id: b1554925-d6a5-488d-8514-8b418191844f
user: diwali?
assistant: I'm not sure what that is, but I'm sure it's a holiday.

User: deepavali vacation?
Nov -23 : 13M Deepavali


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Conversation id: 89437e45-20ae-4465-8c47-416536eb9abc
user: deepavali vacation?
assistant: I'm not sure what that is, but I'll look it up.

User: bye
GoodBye
